### Define ECR repository and model tar file name

In [ ]:
ecr_repo_name = "serverless_container_repo"
model_tar_file_name = "u2net_model.tar.gz"

### Download model file and Create a tar file of model

In [ ]:
!pip install gdown
!python setup_model_weights.py
!tar -czvf $model_tar_file_name u2net.pth

### Upload the tar.gz file to s3  
#### Before running the cell, please update the s3 bucket name

In [ ]:
import boto3
s3_bucket_name = "ENTER YOUR S3 BUCKET NAME HERE"
s3_client = boto3.client('s3')
model_s3_key = "u2net_model/" + model_tar_file_name
result = s3_client.upload_file(model_tar_file_name, s3_bucket_name, model_s3_key)
u2net_model_url = "https://{}.s3.amazonaws.com/{}".format(s3_bucket_name, model_s3_key)
print("Model file successfully uploaded to s3")
print("Model s3 URL: ",u2net_model_url)

### Build docker image for the model and push it to ECR repository

In [ ]:
!bash build_and_push.sh $ecr_repo_name

### Import required libraries

In [ ]:
%%time
import sagemaker
from sagemaker import get_execution_role

runtime = boto3.client(service_name="runtime.sagemaker")

### Initialize Parameters

In [ ]:
sm_client = boto3.client(service_name="sagemaker")

account_id = boto3.client("sts").get_caller_identity()["Account"]
region = boto3.Session().region_name

role = get_execution_role()
print(account_id, region, role)

### Define Model, Configuration and Endpoint names

In [ ]:
u2net_model_name = "serverless-endpoint-u2net-model"
u2net_endpoint_config_name = "serverless-endpoint-u2net-config" 
# If you change the endpoint name here, you'll also need to update it in the lambda function code
u2net_endpoint_name = "serverless-endpoint-u2net"

### Create a model with Model weights file and ECR container image

In [ ]:
u2net_container = "{}.dkr.ecr.{}.amazonaws.com/{}:latest".format(
    account_id, region, ecr_repo_name
)

print("Creating Model: " + u2net_model_name)
print("Container image for the model: " + u2net_container)

containers = {"Image": u2net_container, "ModelDataUrl":u2net_model_url, "Mode": "SingleModel"}

u2net_create_model_response = sm_client.create_model(
    ModelName=u2net_model_name, ExecutionRoleArn=role, Containers=[containers]
)

print("Model created successfully")

### Create Endpoint configuration

In [ ]:
print("Creating Endpoint configuration: " + u2net_endpoint_config_name)

u2net_create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=u2net_endpoint_config_name,
    ProductionVariants=[
        {
            "ModelName": u2net_model_name,
            "VariantName": "test",
            'ServerlessConfig': {
                'MemorySizeInMB': 6144,
                'MaxConcurrency': 2
            }
        }
    ],
)
print("Endpoint Configuration created successfully")

### Create an Endpoint  
#### The endpoint will take about 10-15 minutes for creating

In [ ]:
print("Creating Endpoint: " + u2net_endpoint_name)
u2net_create_endpoint_response = sm_client.create_endpoint(
    EndpointName=u2net_endpoint_name, EndpointConfigName=u2net_endpoint_config_name
)

print("Waiting for {} endpoint to be in service...".format(u2net_endpoint_name))
u2net_waiter = sm_client.get_waiter("endpoint_in_service")
u2net_waiter.wait(EndpointName=u2net_endpoint_name)
print("Endpoint created successfully")

### Create a Lambda Function
#### Enter the IAM role name before running the cell

In [ ]:
lambda_client = boto3.client("lambda")
iam_role_name = "ENTER YOUR IAM ROLE NAME HERE"
iam_role_arn = "arn:aws:iam::{}:role/{}".format(account_id, iam_role_name)
lambda_function_name = 'webapp-host-lambda'

with open("lambda_function.zip", 'rb') as file_data:
    bytes_content = file_data.read()

    response = lambda_client.create_function(
            Code={
                'ZipFile': bytes_content
            },
            FunctionName=lambda_function_name,
            Handler='lambda_function.lambda_handler',
            Publish=True,
            Role=iam_role_arn,
            Runtime='python3.9',
            Timeout=60
        )
    lambda_arn = response['FunctionArn']
print("Lambda function created successfully")

### Create API Gateway

In [ ]:
api_client = boto3.client('apigateway')

rest_api_response = api_client.create_rest_api(
    name='webapp-api',
    binaryMediaTypes=[
        'multipart/form-data',
    ],
    endpointConfiguration={
        'types': [
            'REGIONAL'
        ]
    }
)

rest_api_id = rest_api_response["id"]

# Get the rest api's root id
root_resource_id = api_client.get_resources(
restApiId=rest_api_id
)['items'][0]['id']

# Create an api resource
api_resource = api_client.create_resource(
restApiId=rest_api_id,
parentId=root_resource_id,
pathPart='inference'
)

api_resource_id = api_resource['id']

# Add methods to the rest api resource
api_method = api_client.put_method(
restApiId=rest_api_id,
resourceId=root_resource_id,
httpMethod='POST',
authorizationType='NONE'
)

put_method_res = api_client.put_method_response(
restApiId=rest_api_id,
resourceId=root_resource_id,
httpMethod='POST',
statusCode='200'
)

api_method = api_client.put_method(
restApiId=rest_api_id,
resourceId=api_resource_id,
httpMethod='POST',
authorizationType='NONE'
)

put_method_res = api_client.put_method_response(
restApiId=rest_api_id,
resourceId=api_resource_id,
httpMethod='POST',
statusCode='200',
responseParameters={
    'method.response.header.Content-Type': True
}
)

arn_uri="arn:aws:apigateway:{}:lambda:path/2015-03-31/functions/{}/invocations".format(region,lambda_arn)

put_integration = api_client.put_integration(
restApiId=rest_api_id,
resourceId=root_resource_id,
httpMethod='POST',
type='AWS_PROXY',
integrationHttpMethod='POST',
uri=arn_uri,
credentials=iam_role_arn
)

put_integration_response = api_client.put_integration_response(
restApiId=rest_api_id,
resourceId=root_resource_id,
httpMethod='POST',
statusCode='200',
selectionPattern=''
)



put_integration = api_client.put_integration(
restApiId=rest_api_id,
resourceId=api_resource_id,
httpMethod='POST',
type='AWS',
integrationHttpMethod='POST',
uri=arn_uri,
credentials=iam_role_arn,
requestTemplates={
  "multipart/form-data":"{\"content\":\"$input.body\"}"
},
passthroughBehavior='WHEN_NO_TEMPLATES'
)

put_integration_response = api_client.put_integration_response(
restApiId=rest_api_id,
resourceId=api_resource_id,
httpMethod='POST',
statusCode='200',
selectionPattern='',
responseParameters={
        'method.response.header.Content-Type': "'text/html'"
    },
responseTemplates={
  "text/html":"$input.path('$').body"
}
)

# Deploy the api
stage = 'dev'
deployment = api_client.create_deployment(
restApiId=rest_api_id,
stageName=stage,
)
print("Use following url to see serverless hosted webapp")
print("https://{}.execute-api.{}.amazonaws.com/{}".format(rest_api_id, region, stage))

### Delete the resources

In [ ]:
sm_client.delete_endpoint(EndpointName=u2net_endpoint_name)
sm_client.delete_endpoint_config(EndpointConfigName=u2net_endpoint_config_name)
sm_client.delete_model(ModelName=u2net_model_name)
print('Successfully deleted endpoint related files')

In [ ]:
lambda_client.delete_function(FunctionName=lambda_function_name)

In [ ]:
api_client.delete_rest_api(restApiId=rest_api_id)